<a href="https://colab.research.google.com/github/Samad80/genAI/blob/main/Explainable_PDF_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import os, shutil

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from transformers import pipeline


CHROMA_PATH = "/content/chroma"

# ---------------- Models ----------------
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    pipeline_kwargs={"max_new_tokens": 300, "temperature": 0.2},
)

ANSWER_PROMPT = """
Answer the question using ONLY the context below.

Context:
{context}

Question:
{question}
"""

EXPLANATION_PROMPT = """
Explain WHY the following context was relevant to answering the question.

Question:
{question}

Context:
{context}

Give a short explanation.
"""

db = None  # vector DB (created after upload)

# ---------------- Functions ----------------
def index_pdf(pdf_file):
    global db

    if pdf_file is None:
        return "❌ No PDF uploaded."

    # Clear old DB
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Gradio gives a FILE PATH (new versions)
    pdf_path = pdf_file

    # Load PDF
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # Split
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=150,
    )
    chunks = splitter.split_documents(documents)

    # Create DB
    db = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=CHROMA_PATH,
    )
    db.persist()

    return f"✅ Indexed PDF successfully ({len(chunks)} chunks)"


def ask_question(question):
    global db

    if db is None:
        return "⚠️ Please upload and index a PDF first.", "", ""

    results = db.similarity_search_with_relevance_scores(question, k=3)

    context = "\n\n---\n\n".join([doc.page_content for doc, _ in results])

    answer = llm.invoke(
        ChatPromptTemplate.from_template(ANSWER_PROMPT).format(
            context=context, question=question
        )
    )

    explanation = llm.invoke(
        ChatPromptTemplate.from_template(EXPLANATION_PROMPT).format(
            context=context, question=question
        )
    )

    sources = []
    for doc, score in results:
        page = doc.metadata.get("page", "N/A")
        source = os.path.basename(doc.metadata.get("source", "PDF"))
        sources.append(f"{source} — page {page}, relevance {score:.2f}")

    return answer, explanation, "\n".join(sources)

# ---------------- UI ----------------
with gr.Blocks() as ui:
    gr.Markdown("## 📘 Explainable PDF Question Answering")

    pdf_input = gr.File(
        label="Upload PDF",
        file_types=[".pdf"],
        type="filepath",   # ⭐ THIS IS CRITICAL
    )

    index_btn = gr.Button("📥 Index PDF")
    status = gr.Textbox(label="Status")

    index_btn.click(
        fn=index_pdf,
        inputs=pdf_input,
        outputs=status,
    )

    question = gr.Textbox(label="Ask a question")
    answer = gr.Textbox(label="📘 Answer")
    explanation = gr.Textbox(label="🧠 Why this answer?")
    sources = gr.Textbox(label="📄 Sources")

    question.submit(
        fn=ask_question,
        inputs=question,
        outputs=[answer, explanation, sources],
    )

ui.launch(share=True)


/tmp/ipython-input-1228811065.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://428d848b9e58441685.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
